In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Things needs from cco file
### A dataframe with
- nVessels rows and in the columns
    - node in, node out
    - Radius
    - [Optional] Flow computed by cco 
    - Daughter of current vessels row number
    - A branch number depending on the inlet they are attached to


In [3]:
def ReadCCO(CCOFileName):
    
    tmpDFVessels = pd.DataFrame([], columns=['Radius', 'ParentIdx', 'BranchesIdx','Flow','Stage', 'Inlet', 'TerminalCap'])
    tmpDFNodes  = pd.DataFrame([], columns=['xProx', 'xDist'])
    
    with open(CCOFileName) as f:
        
        f.readline()
        f.readline()
        f.readline() 
        
        f.readline()
        nVessels = int(f.readline())
        print(f'Found {nVessels} in the initial tree.')\
       
        for i in range(nVessels):
            vessel = f.readline().split() 
            idx, r, flow, stage = int(vessel[0]), max(float(vessel[12]), 4e-4), float(vessel[13]), int(vessel[-1])
            xProx, xDist        = np.array(vessel[1:4]).astype(float), np.array(vessel[4:7]).astype(float)
            
            # Keep vessel only if within FOV
            if stage>-2:
                tmpDFVessels.at[idx,'Radius'] = r
                tmpDFVessels.at[idx,'Flow']   = flow
                tmpDFVessels.at[idx,'Stage']  = stage
                tmpDFNodes.at[idx] = [xProx, xDist]
        
        f.readline()
        f.readline()
        idxToKeep = tmpDFVessels.index
        for i in range(nVessels):
            vessel = f.readline().split()
            idx, parentIdx, branchesIdx = int(vessel[0]), int(vessel[1]), [int(b) for b in vessel[2:]]
            
            if idx in idxToKeep:
                tmpDFVessels.at[idx, 'ParentIdx']  = parentIdx
                tmpDFVessels.at[idx, 'BranchesIdx'] = branchesIdx
                
                if not parentIdx in idxToKeep:
                    tmpDFVessels.at[idx, 'Inlet'] = True
                elif not branchesIdx:
                    tmpDFVessels.at[idx, 'TerminalCap'] = True
                else:
                    tmpDFVessels.at[idx, 'Inlet'] = False
                    tmpDFVessels.at[idx, 'TerminalCap'] = False
            
    # Create the nodes list and connection table
    nVessels   = tmpDFVessels.shape[0]
    inletIdx   = tmpDFVessels[tmpDFVessels.Inlet==True].index 
    termCapIdx = tmpDFVessels[tmpDFVessels.TerminalCap==True].index
    nInlets, nTermCap = len(inletIdx), len(termCapIdx)
    arterioles = np.arange(nInlets)
    np.random.shuffle(arterioles)
    arterioles = arterioles[:int(round(nInlets/2.0))]

    
    print(f'Kept {nVessels} in the final tree, with {nInlets} inlets and {nTermCap} terminal capillaries.')
    print(f'Branch numbers {arterioles}, will be counted as arterioles.')
    
    df = pd.DataFrame([], columns=['NodeIn', 'NodeOut', 'Radius', 'Flow', 'BranchNumber','Arteriole'])
    
    def BuildNodeListAndAssignBranchNumber(inletIdx, currentNodeIdx, branchNumber):
            
            if branchNumber in arterioles:
                nodeIn, nodeOut = currentNodeIdx, currentNodeIdx+1
            else:
                nodeIn, nodeOut = currentNodeIdx+1, currentNodeIdx
            
            df.at[inletIdx] = [nodeIn, nodeOut,
                               tmpDFVessels.at[inletIdx,'Radius'],
                               tmpDFVessels.at[inletIdx,'Flow'],
                               branchNumber,
                               branchNumber in arterioles]

            for branch in tmpDFVessels.at[inletIdx,'BranchesIdx']:
                BuildNodeListAndAssignBranchNumber(branch, currentNodeIdx+1, branchNumber)
    
    currentNodeIdx = 0
    
    for branchNumber, inlet in enumerate(inletIdx):
        
        BuildNodeListAndAssignBranchNumber(inlet, currentNodeIdx, branchNumber)
        currentNodeIdx = df.NodeOut.max()
 
    
    ''' Create the connection between terminal capillaries. 
        Terminal arteriolar capillaries are linked to the 
        closest terminal venous capillary.
    '''
    venousCap = df[df.Arteriole==False].index
    venousCapCandidates = [int(cap) for cap in venousCap] 
    arteriolarCap = df[df.Arteriole==True].index
    usedAllVenousCap = False
    newVesselIdx = df.index.max()

    for artCapIdx in arteriolarCap:
        
        # Find closest terminal node in the venous tree 
        if not venousCapCandidates:
            venousCapCandidates = [int(cap) for cap in venousCap] # If we have exhausted
                                                                  # the terminal capillaries in the venous tree, 
            usedAllVenousCap = True         # we allow more than a single connection to each  
        nodeOutArt = tmpDFNodes.at[artCapIdx,'xDist'] 
        
        candidatesNodes = tmpDFNodes.loc[venousCapCandidates].xProx
        candidatesIdx   = candidatesNodes.index
        positions = np.array([list(x) for x in candidatesNodes.values])
        distances = np.linalg.norm(nodeOutArt-positions, axis=1)
        chosenCapIdx = candidatesIdx[distances.argmin()]
        venousCapCandidates.remove(chosenCapIdx) # Remove the chosen node from the list
        
        # Create a link between the two nodes
        newVesselIdx +=1
        df.at[newVesselIdx] = [df.at[artCapIdx,'NodeOut'], df.at[chosenCapIdx,'NodeIn'],
                               4.0e-4, 0.0, -1, None] # Branch number = -1 for connecting vessels added at this stage
        
        
    if not usedAllVenousCap:
        for venCapIdx in venousCapCandidates:
            
            nodeInVen = tmpDFNodes.at[venCapIdx,'xProx']
            candidatesNodes = tmpDFNodes.loc[arteriolarCap].xDist
            candidatesIdx = candidatesNodes.index
            positions = np.array([list(x) for x in candidatesNodes.values])
            distances = np.linalg.norm(nodeOutArt-positions, axis=1)
            newVesselIdx+=1
            df.at[newVesselIdx] = [df.at[chosenCapIdx,'NodeOut'], df.at[venCapIdx,'NodeIn'],
                                   4.0e-4, 0.0, -2, None] # Branch number = -2 for connecting vessels added at this stage

        

    
    ## Connection table
    df = df.reset_index(drop=False)
    nNodes   = max(df.NodeIn.max(), df.NodeOut.max()) + 1 
    nVessels = df.shape[0] 
    C = np.zeros((nNodes,nVessels))
    for i,row in df.iterrows():
        nodeIn, nodeOut = row.NodeIn, row.NodeOut
        C[nodeIn,i]  = 1
        C[nodeOut,i] = -1
        
    
    
    return C, df

In [4]:
C, df = ReadCCO('20TerminalVessels.cco')
df

Found 154 in the initial tree.
Kept 105 in the final tree, with 7 inlets and 27 terminal capillaries.
Branch numbers [3 5 0 2], will be counted as arterioles.


,index,NodeIn,NodeOut,Radius,Flow,BranchNumber,Arteriole
0,23,0,1,0.002012,1.37931,0,True
1,24,1,2,0.002012,1.37931,0,True
2,120,2,3,0.001236,0.344828,0,True
3,121,2,3,0.00182,1.034483,0,True
4,25,3,4,0.00182,1.034483,0,True
...,...,...,...,...,...,...,...
154,203,64,42,0.0004,0.0,-1,None
155,204,65,41,0.0004,0.0,-1,None
156,205,66,82,0.0004,0.0,-1,None
157,206,66,45,0.0004,0.0,-1,None


In [5]:
print(df[df.BranchNumber==-1].shape[0], df[df.BranchNumber==-2].shape[0])

54 0


In [6]:
nodeList = np.zeros(C.shape[0])
for i,row in df.iterrows():
    nodeIn, nodeOut = row.NodeIn, row.NodeOut
    nodeList[nodeIn]+=1
    nodeList[nodeOut]+=1
print(C.shape, 'Too many segments for too few nodes. Must have wrote over existing nodes somehow.')

(83, 159) Too many segments for too few nodes. Must have wrote over existing nodes somehow.
